In [272]:
# importing necessary libraries
import requests 
from bs4 import BeautifulSoup 
import csv 
import pandas as pd 
import re

# creating a data frame 
df = pd.read_csv("data.csv") 

# print(df.head())
print(df['Pull Request'])
print(type(df))

0               https://github.com/apache/juneau/pull/16
1      https://github.com/apache/incubator-druid/pull...
2      https://github.com/apache/incubator-gobblin/pu...
3              https://github.com/apache/ambari/pull/760
4               https://github.com/apache/nutch/pull/234
                             ...                        
351           https://github.com/facebook/jest/pull/6650
352          https://github.com/facebook/react/pull/4447
353           https://github.com/facebook/jest/pull/7385
354           https://github.com/facebook/jest/pull/2621
355          https://github.com/facebook/react/pull/4787
Name: Pull Request, Length: 356, dtype: object
<class 'pandas.core.frame.DataFrame'>


In [273]:
apacheDataSet = df[df['Repo'].str.contains('/apache')]

In [274]:
mozillaDataSet = df[df['Repo'].str.contains('/mozilla')]

In [275]:
googleDataSet = df[df['Repo'].str.contains('/google')]

In [276]:
facebookDataSet = df[df['Repo'].str.contains('/facebook')]
# display(googleDataSet)
# display(googleDataSet.count())

In [277]:
pullRequestList=apacheDataSet['Pull Request'].values.tolist()

In [278]:
display(len(pullRequestList))

88

In [279]:
titleData = {'title': [],'comment':[]}
df_marks = pd.DataFrame(titleData)
df_marks['title']

Series([], Name: title, dtype: float64)

In [319]:
dictw = {'title':[], 
        'comment':[]
       } 
df_marks = pd.DataFrame(dictw) 

In [320]:
commitsList=[]
filesChangedList = []
green = []
red = []

for i in range(len(pullRequestList)):
    URL = pullRequestList[i]
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html.parser') 
    mainTitle = soup.find('h1', attrs = {'class':'gh-header-title mb-2 lh-condensed f1 mr-0 flex-auto break-word'})
    comment = soup.find('div',attrs={'class':'edit-comment-hide'})
    
    commits = soup.find('span', attrs = {'class':'js-updateable-pull-request-commits-count Counter'})
    files_changed = soup.find('span', attrs = {'id':'files_tab_counter'})
    
    if commits:
        commitsList.append(commits.get_text())
    else:
        commitsList.append("")
        
    if files_changed:
        filesChangedList.append(files_changed.get_text())
    else:
        filesChangedList.append("")
    
    code_churn_data = soup.find('span', attrs = {'id':'diffstat'})
#     print(code_churn_data)
    if code_churn_data:
        for i in code_churn_data.findAll('span', attrs = {'class':'text-green'}):
#             print(re.sub(r'\D', '', i.text))
            green.append(re.sub(r'\D', '', i.text))
        for j in code_churn_data.findAll('span', attrs = {'class':'text-red'}):
            red.append(re.sub(r'\D', '', j.text))
#             print(re.sub(r'\D', '', j.text))
    else:
        green.append(0)
        red.append(0)

        
    heading=""
    if mainTitle:
        for row in mainTitle.findAll('span', attrs = {'class':'js-issue-title'}):
            heading=row.get_text()
            for roww in comment.findAll('td', attrs = {'class':'d-block comment-body markdown-body js-comment-body'}):
                finalComment=""
                for rowww in roww.find_all(['p', 'li']):
                    finalComment=finalComment+rowww.get_text()+'.'
            df_marks.loc[len(df_marks.index)] = [heading, finalComment] 


In [321]:
print(commitsList)
print(len(commitsList))
df_marks['num_commits'] = commitsList

['4', '2', '19', '3', '2', '1', '3', '4', '1', '1', '1', '1', '1', '9', '1', '2', '2', '1', '1', '4', '5', '3', '3', '3', '1', '1', '8', '2', '4', '3', '1', '2', '3', '32', '2', '2', '2', '1', '3', '2', '1', '2', '2', '1', '1', '1', '1', '3', '4', '1', '1', '1', '2', '2', '1', '2', '1', '4', '1', '1', '1', '1', '4', '4', '1', '1', '1', '4', '1', '1', '4', '5', '2', '1', '5', '1', '3', '1', '7', '1', '1', '1', '1', '9', '3', '3', '2', '8']
88


In [322]:
print(filesChangedList)
print(len(filesChangedList))
df_marks['files_changed'] = filesChangedList

['7', '8', '18', '2', '7', '2', '29', '8', '16', '2', '', '', '', '10', '1', '', '2', '', '', '2', '', '', '', '', '11', '2', '7', '', '', '', '1', '', '', '', '104', '104', '', '', '7', '5', '2', '', '', '', '12', '', '', '', '', '2', '2', '', '', '', '2', '', '2', '', '', '', '3', '1', '4', '', '', '', '1', '', '2', '1', '', '', '3', '', '6', '', '', '2', '', '2', '5', '5', '1', '2', '21', '6', '2', '']
88


In [323]:
greenlist = list(map(int, green)) 
redlist = list(map(int, red)) 
print(redlist)
print(greenlist)

[1, 19, 114, 5, 11, 1, 115, 17, 126, 46, 0, 0, 0, 9, 1, 0, 2, 0, 0, 6, 0, 0, 0, 0, 19, 10, 26, 0, 0, 0, 3, 0, 0, 0, 186, 186, 0, 0, 0, 54, 9, 0, 0, 0, 178, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 6, 1, 14, 0, 0, 0, 12, 0, 0, 1, 0, 0, 51, 0, 71, 0, 0, 4, 0, 1, 29, 29, 3, 1, 32, 71, 4, 0]
[386, 233, 1270, 23, 17, 14, 1376, 334, 2065, 66, 0, 0, 0, 1076, 1, 0, 3, 0, 0, 8, 0, 0, 0, 0, 69, 25, 42, 0, 0, 0, 3, 0, 0, 0, 580, 580, 0, 0, 758, 73, 33, 0, 0, 0, 169, 0, 0, 0, 0, 11, 161, 0, 0, 0, 10, 0, 15, 0, 0, 0, 20, 5, 78, 0, 0, 0, 12, 0, 37, 1, 0, 0, 117, 0, 145, 0, 0, 32, 0, 33, 37, 37, 3, 20, 1032, 403, 23, 0]


In [324]:
res_list = [] 
for i in range(0, len(green)): 
    res_list.append(greenlist[i] + redlist[i]) 

In [325]:
df_marks['code_churn'] = res_list

In [326]:
df_marks

,title,comment,num_commits,files_changed,code_churn
0,\n Added LN ST PE vars to svl-core\n,This is related to last mail thread..$LN = for...,4,7,387
1,\n Add SQL REGEXP_EXTRACT function; add...,Syntax is the same as Hive/Presto's REGEXP_EXT...,2,8,252
2,\n Initial pull request for the Gobblin...,This pull request contains the changes for Mut...,19,18,1384
3,\n [AMBARI-23334] Using the proper regu...,The regular expression we use to validate --ss...,3,2,28
4,\n NUTCH-2394 Fix of bugs detected by s...,String.trim() without assignment.avoid strings...,2,7,28
...,...,...,...,...,...
83,\n CB-14198: (all) Fix bug when running...,This is an improved version of #11 PR created ...,9,2,21
84,\n Cpp client: add PatternMultiTopicsCo...,In PR #1279 and #1298 we added regex based sub...,3,21,1064
85,\n SLING-7741 - org.apache.sling.xss.im...,implemented the URI grammar from RFC3986 as a ...,3,6,474
86,\n [sql lab] extract Hive error message...,So pyhive returns an exception object with a s...,2,2,27


In [327]:
comment_list=df_marks['comment'].values.tolist()

In [328]:
title_list=df_marks['title'].values.tolist()

In [329]:
rootCause=[]
# regExString=["regex","regular expression"]
# apiURLString=["api","url"]
for index in range(len(comment_list)):
    if "regex" in comment_list[index] or "regex" in title_list[index]: 
        rootCause.append("regEx")
    elif "regular expression" in comment_list[index] or "regex" in title_list[index]:
        rootCause.append("regEx")
    elif "URL" in comment_list[index] or "regex" in title_list[index]:
        rootCause.append("API")
    elif "API" in comment_list[index] or "regex" in title_list[index]:
        rootCause.append("API")
    else:
        rootCause.append("Other")

In [330]:
df_marks['Root Cause'] = rootCause 

In [331]:
df_marks

,title,comment,num_commits,files_changed,code_churn,Root Cause
0,\n Added LN ST PE vars to svl-core\n,This is related to last mail thread..$LN = for...,4,7,387,regEx
1,\n Add SQL REGEXP_EXTRACT function; add...,Syntax is the same as Hive/Presto's REGEXP_EXT...,2,8,252,regEx
2,\n Initial pull request for the Gobblin...,This pull request contains the changes for Mut...,19,18,1384,regEx
3,\n [AMBARI-23334] Using the proper regu...,The regular expression we use to validate --ss...,3,2,28,regEx
4,\n NUTCH-2394 Fix of bugs detected by s...,String.trim() without assignment.avoid strings...,2,7,28,regEx
...,...,...,...,...,...,...
83,\n CB-14198: (all) Fix bug when running...,This is an improved version of #11 PR created ...,9,2,21,regEx
84,\n Cpp client: add PatternMultiTopicsCo...,In PR #1279 and #1298 we added regex based sub...,3,21,1064,regEx
85,\n SLING-7741 - org.apache.sling.xss.im...,implemented the URI grammar from RFC3986 as a ...,3,6,474,regEx
86,\n [sql lab] extract Hive error message...,So pyhive returns an exception object with a s...,2,2,27,regEx


In [332]:
manifestation=[]
newFeatureStringList=["add","ADD"]
incorrectBehaviourStringList=["proper","modified","fixed","fix","change","changing","changes","expanding regex","expand"]
compileErrorStringList=["detected","reworked","confusing"]
badsmellrStringList=["refactor","update","updating","parse","Replace","better"]
incorrectComputationStringList=["assertion"]

for index in range(len(comment_list)):
    if any(x in comment_list[index] for x in newFeatureStringList) or any(x in title_list[index] for x in newFeatureStringList):
        manifestation.append("new feature")
    elif any(x in comment_list[index] for x in incorrectComputationStringList) or any(x in title_list[index] for x in incorrectComputationStringList):
         manifestation.append("incorrect computation")
    elif any(x in comment_list[index] for x in badsmellrStringList) or any(x in title_list[index] for x in badsmellrStringList):
         manifestation.append("bad smell")
    elif any(x in comment_list[index] for x in compileErrorStringList) or any(x in title_list[index] for x in compileErrorStringList):
        manifestation.append("compile error")
    elif any(x in comment_list[index] for x in incorrectBehaviourStringList) or any(x in title_list[index] for x in incorrectBehaviourStringList):
         manifestation.append("incorrect behavior")
    else:
        manifestation.append("other failure")

In [333]:
df_marks['Manifestation'] = manifestation 

In [334]:
df_marks

,title,comment,num_commits,files_changed,code_churn,Root Cause,Manifestation
0,\n Added LN ST PE vars to svl-core\n,This is related to last mail thread..$LN = for...,4,7,387,regEx,other failure
1,\n Add SQL REGEXP_EXTRACT function; add...,Syntax is the same as Hive/Presto's REGEXP_EXT...,2,8,252,regEx,new feature
2,\n Initial pull request for the Gobblin...,This pull request contains the changes for Mut...,19,18,1384,regEx,new feature
3,\n [AMBARI-23334] Using the proper regu...,The regular expression we use to validate --ss...,3,2,28,regEx,incorrect behavior
4,\n NUTCH-2394 Fix of bugs detected by s...,String.trim() without assignment.avoid strings...,2,7,28,regEx,compile error
...,...,...,...,...,...,...,...
83,\n CB-14198: (all) Fix bug when running...,This is an improved version of #11 PR created ...,9,2,21,regEx,new feature
84,\n Cpp client: add PatternMultiTopicsCo...,In PR #1279 and #1298 we added regex based sub...,3,21,1064,regEx,new feature
85,\n SLING-7741 - org.apache.sling.xss.im...,implemented the URI grammar from RFC3986 as a ...,3,6,474,regEx,incorrect behavior
86,\n [sql lab] extract Hive error message...,So pyhive returns an exception object with a s...,2,2,27,regEx,other failure


In [335]:
category=[]
acceptingInvalidStringList=["Bugfix for regex","Fix integration test script","SSP mail form"]
incorrectExtractionStringList=["StringExpander regex","fix whitelist","Fix greedy regex"]
dataProcessingStringList=["Add helper script","Add sec","Enabling extraction","Add custom context"]
badSmellStringList=["Refactor","Rename","Optimize"]
regexConfigStringList=["configurable","config"]
regexLikeImplStringList=["REGEXP_EXTRACT","Regex replacement","regex filter"]
rejectingValidStringList=["better regex","RegEx pattern should be updated","replaces the regex"]
codeSmelldStringList=["regex pattern","replace by regular","Remove"]


for index in range(len(comment_list)):
    if any(x in comment_list[index] for x in acceptingInvalidStringList) or any(x in title_list[index] for x in acceptingInvalidStringList):
        category.append("accepting invalid strings(T2)")
    elif any(x in comment_list[index] for x in incorrectExtractionStringList) or any(x in title_list[index] for x in incorrectExtractionStringList):
        category.append("incorrect extraction")
    elif any(x in comment_list[index] for x in dataProcessingStringList) or any(x in title_list[index] for x in dataProcessingStringList):
        category.append("data processing")
    elif any(x in comment_list[index] for x in badSmellStringList) or any(x in title_list[index] for x in badSmellStringList):
        category.append("bad smells")
    elif any(x in comment_list[index] for x in regexConfigStringList) or any(x in title_list[index] for x in regexConfigStringList):
        category.append("regex configuration entry")
    elif any(x in comment_list[index] for x in regexLikeImplStringList) or any(x in title_list[index] for x in regexLikeImplStringList):
        category.append("regex like implementation")
    elif any(x in comment_list[index] for x in rejectingValidStringList) or any(x in title_list[index] for x in rejectingValidStringList):
        category.append("rejecting valid strings (T1)")
    elif any(x in comment_list[index] for x in codeSmelldStringList) or any(x in title_list[index] for x in codeSmelldStringList):
        category.append("code smells")
    else:
        category.append(" ")

In [336]:
df_marks['Category'] = category 

In [337]:
df_marks.to_csv(r'scrapedData.csv', sep=',', mode='a')